In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import glob
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import StepLR
import torch
import torchvision
from torchvision import transforms


c:\Users\Shaan\miniconda3\envs\pytorc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Using device: cuda
NVIDIA GeForce RTX 3070


In [4]:
def check_file_size(path):
    return os.stat(path).st_size < 10000000 # 10 MB

In [5]:
data_dir_train = 'D:\ML for Bio\\Binary\\train'
data_dir_val = 'D:\ML for Bio\\Binary\\val'
data_dir_test = 'D:\ML for Bio\\Binary\\test'

transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.RandomCrop((400, 400)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset_train = torchvision.datasets.ImageFolder(data_dir_train, transform=transform, target_transform=None, is_valid_file=lambda path: check_file_size(path))
dataset_val = torchvision.datasets.ImageFolder(data_dir_val, transform=transform, target_transform=None, is_valid_file=lambda path: check_file_size(path))
dataset_test = torchvision.datasets.ImageFolder(data_dir_test, transform=transform, target_transform=None, is_valid_file=lambda path: check_file_size(path))


In [6]:
batch_size = 8
shuffle = True  
num_workers = 16  
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [7]:
dataloader_train

In [8]:
def test(model, dataloader, criterion, device):
    model.eval()
    model.to(device)

    total_loss = 0.0
    num_correct = 0
    n = 0
    
    with torch.no_grad():
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc='1/1', leave=True)

        for batch_idx, (images, labels) in progress_bar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            num_correct += torch.sum(preds == labels)
            n += len(labels)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            #if batch_idx%5 == 0:
                #print("Loss at batch " + str(batch_idx) + ": " + str(loss.item()))
            
            del images, labels

    avg_loss = total_loss / (batch_idx + 1)
    accuracy = num_correct / n * 100

    print("Loss: " + str(avg_loss))
    print("Accuracy: " + str(accuracy))

    return avg_loss, accuracy

In [9]:
def train(model, dataloader, criterion, optimizer, device, epochs, val_dataset, scheduler=None, num_batches=1):
    losses = []
    accuracies = []
    val_losses = []
    val_acc = []
    best_val = 0
    for epoch in range(epochs):
        model.train()
        model.to(device)
        total_loss = 0.0
        num_correct = 0
        n = 0
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Epoch {epoch + 1}/{epochs}', leave=True)

        optimizer.zero_grad()

        for batch_idx, (images, labels) in progress_bar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            num_correct += torch.sum(preds == labels)
            n += len(labels)

            loss = criterion(outputs, labels)
            loss.backward()

            if batch_idx%batch_size == 0 or batch_idx == len(dataloader)-1:
                optimizer.step()
                optimizer.zero_grad()

            total_loss += loss.item()
            del images, labels
            torch.cuda.empty_cache()

        if scheduler is not None:
            scheduler.step()

            
        avg_loss_val, accuracy_val = test(model, val_dataset, criterion, device)
        val_acc.append(accuracy_val)
        val_losses.append(avg_loss_val)

        if accuracy_val > best_val:
            best_val = accuracy_val
            best_model_wts = copy.deepcopy(model.state_dict())


        avg_loss = total_loss / (batch_idx + 1)
        accuracy = num_correct/n * 100
        losses.append(avg_loss)
        accuracies.append(accuracy)
        print("Loss at epoch " + str(epoch) + ": " + str(avg_loss))
        print("Accuracy at epoch " + str(epoch) + ": " + str(accuracy))

    model.load_state_dict(best_model_wts)
    return model, losses, accuracies, val_losses, val_acc

In [ ]:
model1 = torchvision.models.efficientnet_b0(weights='DEFAULT')
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='DEFAULT')
model3 = torchvision.models.resnet50(weights = 'DEFAULT')
models = [model1, model2, model3]
for model in models:
    print("============================================================")
    print("============================TRAIN===========================")
    print("============================================================")
    criterion = nn.CrossEntropyLoss()
    epochs = 30
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.75)
    num_batches = 8
    model.to(device)
    model, losses, accuracies, val_losses, val_acc = train(model, dataloader_train, criterion, optimizer, device, epochs, dataloader_val, scheduler, num_batches)
    print("============================================================")
    print("============================TEST============================")
    print("============================================================")
    avg_loss, accuracy = test(model, dataloader_test, criterion, device)
    del model
    torch.cuda.empty_cache()        
